In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser
from datetime import datetime
import json

In [ ]:
file = "MasterData_Wilson.xlsx"
pd_file = pd.read_excel(file)
bakery_df = pd.DataFrame(pd_file)
bakery_df.head()

In [ ]:
bakery_df.columns

In [ ]:
bakery_df['Unnamed: 29'].value_counts()

In [ ]:
bakery_df.loc[(bakery_df['Unnamed: 29'] == 'Sat') | (bakery_df['Unnamed: 29'] == 2), :]

In [ ]:
bakery_df['Unnamed: 30'].value_counts()

In [ ]:
bakery_df = bakery_df[['date', 'time', 'year', 'MonthN', 'Month', 'WeekN', 'Day', 'WeekD', 'H',
       'Min', '30min', 'Rtime', '30time', 'H.5', 'Operation', 'Line Item',
       'Department', 'Category', 'Supplier', 'Supplier Code', 'Cost', 'Price',
       'Quantity', 'Modifiers', 'Subtotal', 'Discounts', 'Net Total', 'Tax',
       'Total Due']]

In [ ]:
bakery_df.head()

In [ ]:
bakery_df.dtypes

In [ ]:
bakery_df['year'] = bakery_df['year'].fillna(0.0).astype(int)

In [ ]:
bakery_df['year'].value_counts()

In [ ]:
bakery_df.loc[bakery_df['year'] == 0, :]

In [ ]:
bakery_df = bakery_df.drop(index=416580)
bakery_df.loc[bakery_df['year'] == 0, :]

In [ ]:
bakery_df['MonthN'] = bakery_df['MonthN'].fillna(0.0).astype(int)

In [ ]:
bakery_df['MonthN'].value_counts()

In [ ]:
bakery_df['WeekN'] = bakery_df['WeekN'].fillna(0.0).astype(int)
bakery_df['WeekN'].value_counts()

In [ ]:
bakery_df['Day'] = bakery_df['Day'].fillna(0.0).astype(int)
bakery_df['Day'].value_counts()

In [ ]:
bakery_df.head()

In [ ]:
cln_bakery_df = bakery_df[['time', 'WeekN', 'WeekD', 'Rtime', 'Operation', 'Line Item', 'Department', \
                           'Category', 'Supplier', 'Supplier Code', 'Cost', 'Price','Quantity', \
                           'Modifiers', 'Subtotal', 'Discounts', 'Net Total', 'Tax', 'Total Due']]

In [ ]:
cln_bakery_df.head()

In [ ]:
cln_bakery_df['Operation'].value_counts()

In [ ]:
cln_bakery_df['Supplier'].value_counts()

In [2]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [15]:
damen_reviews = []
belmont_reviews = []
logan_reviews = []

damen_bakeries = ["https://www.yelp.com/biz/la-boulangerie-chicago-4?sort_by=date_desc", \
                  "https://www.yelp.com/biz/la-boulangerie-chicago-4?start=20&sort_by=date_desc", \
                  "https://www.yelp.com/biz/la-boulangerie-chicago-4?start=40&sort_by=date_desc", \
                  "https://www.yelp.com/biz/la-boulangerie-chicago-4?start=60&sort_by=date_desc"]
belmont_bakeries = ['https://www.yelp.com/biz/la-boulangerie-chicago-2?sort_by=date_desc', \
                       'https://www.yelp.com/biz/la-boulangerie-chicago-2?start=20&sort_by=date_desc', \
                       'https://www.yelp.com/biz/la-boulangerie-chicago-2?start=40&sort_by=date_desc', \
                       'https://www.yelp.com/biz/la-boulangerie-chicago-2?start=60&sort_by=date_desc', \
                       'https://www.yelp.com/biz/la-boulangerie-chicago-2?start=80&sort_by=date_desc', \
                       'https://www.yelp.com/biz/la-boulangerie-chicago-2?start=100&sort_by=date_desc', \
                       'https://www.yelp.com/biz/la-boulangerie-chicago-2?start=120&sort_by=date_desc', \
                       'https://www.yelp.com/biz/la-boulangerie-chicago-2?start=140&sort_by=date_desc', \
                       'https://www.yelp.com/biz/la-boulangerie-chicago-2?start=160&sort_by=date_desc']
logan_bakeries = ['https://www.yelp.com/biz/la-boulangerie-chicago-8?sort_by=date_desc', \
                 'https://www.yelp.com/biz/la-boulangerie-chicago-8?start=20&sort_by=date_desc']


In [16]:
def yelp_bakery_lookup(bakery_list, review_list):
    for link in bakery_list:
        bakery_response = requests.get(link)
        bakery_soup = bs(bakery_response.text, 'html.parser')
        bakery_results = bakery_soup.body.find_all('div', class_='review--with-sidebar')
        for review in bakery_results:
            user_info = []
            user_stats = []
            user_name = '' 
            user_loc = ''
            user_friends = ''
            user_reviews = ''
            user_photos = ''
            review_info = []
            review_stars = ''
            review_date = ''
            review_text = ''
            user_check_ins = ''
            elite = ''
            elite_status = ''
            photo = []
            photo_links = []
            review_data = []
            previous_review = []
            previous_rating = []

            user_information = {}

            try:
                user_info = review.find('ul', class_='user-passport-info')
                user_stats = review.find('ul', class_='user-passport-stats')
                review_info = review.find('div', class_='review-content')
                review_footer = review.find('div', class_='review-footer')

                try:
                    user_name = user_info.find('li', class_='user-name').find('a').text.strip()
                    user_profile_link = user_info.find('a', class_='user-display-name').attrs['href']
                    user_profile_link = "https://www.yelp.com" + user_profile_link
                    user_loc = user_info.find('li', class_='user-location').find('b').text.strip()
                    review_stars = review_info.find('div', class_='i-stars').attrs['title']
                    review_stars = int(float(review_stars.split()[0]))
                    review_date = review_info.find('span', class_='rating-qualifier').text.strip()
                    review_text = review_info.find('p').text
                except AttributeError as error:
                    print(error)

                try:
                    user_friends = user_stats.find('li', class_='friend-count').find('b').text.strip()
                except AttributeError as error:
                    if user_stats:
                        user_friends = '0'

                try:
                    user_reviews = user_stats.find('li', class_='review-count').find('b').text.strip()
                except AttributeError as error:
                    if user_stats:
                        user_reviews = '0'

                try:
                    user_photos = user_stats.find('li', class_='photo-count').find('b').text.strip()
                except AttributeError as error:
                    if user_stats:
                        user_photos = '0'

                try:
                    elite = user_stats.find('li', class_='is-elite')
                    if elite:
                        elite_status = 'elite'
                    else:
                        elite_status = 'not elite'
                except AttributeError as error:
                    print(error)

                try:
                    user_check_ins = review_info.find('li', class_='review-tags_item').find('span').text.strip()
                    user_check_ins = user_check_ins.split()[0]
                except AttributeError as error:
                    if review_info:
                        user_check_ins = '0'

                try:
                    photos = review_info.find('ul', class_='photo-box-grid')
                    if photos:
                        try:
                            picture = photos.find_all('div', class_='photo-box')
                            for pic in picture:
                                pic_link = ''
                                pic_link = pic.find('img').attrs['data-async-src']
                                photo_links.append(pic_link)
                        except Exception as error:
                            print(error)

                except AttributeError as error:
                    if review_info:
                        photo_links = []

                try:
                    review_options = review_footer.find_all('li', class_='vote-item')
                    review_data = {
                        'review_type' : [],
                        'review_count' : []
                    }
                    for option in review_options:
                        review_type = ''
                        review_count = ''
                        review_type = option.find('a').find('span', class_='vote-type').text.strip()
                        review_count = option.find('a').find('span', class_='count').text.strip()
                        if review_count == '':
                            review_count = '0'
                        review_data['review_type'].append(review_type)
                        review_data['review_count'].append(review_count)
                except AttributeError as error:
                    print(error)

                try:
                    previous_review = review.find('div', class_='previous-review')
                    if previous_review:
                        prev_star = previous_review.find('div', class_='i-stars').attrs['title']
                        prev_star = int(float(prev_star.split()[0]))
                        prev_date = previous_review.find('span', class_='rating-qualifier').text.strip()
                        prev_text = previous_review.find('span', class_=['hidden']).text.strip()
                        prev_review_options = previous_review.find_all('li', class_='vote-item')
                        prev_review_data = {
                            'review_type' : [],
                            'review_count' : []
                        }
                        for option in prev_review_options:
                            review_type = ''
                            review_count = ''
                            review_type = option.find('a').find('span', class_='vote-type').text.strip()
                            review_count = option.find('a').find('span', class_='count').text.strip()
                            if review_count == '':
                                review_count = '0'
                            prev_review_data['review_type'].append(review_type)
                            prev_review_data['review_count'].append(review_count)
                        previous_rating = {
                            'stars' : prev_star,
                            'review_date' : prev_date,
                            'review_text' : prev_text,
                            'review_feedback' : prev_review_data
                        }
                except AttributeError as error:
                    previous_review = []
                    print(error)

                if (user_name and user_loc and user_friends and user_reviews and user_photos and review_stars \
                    and review_date and user_check_ins and user_profile_link and review_text):
                    try:
                        user_check_ins = int(user_check_ins)
                    except ValueError as error:
                        user_check_ins = 0
                    user_information = {
                        'user_name' : user_name,
                        'user_location' : user_loc,
                        'user_link' : user_profile_link,
                        'user_friends' : int(user_friends),
                        'user_reviews' : int(user_reviews),
                        'user_photos' : int(user_photos),
                        'stars' : review_stars,
                        'review_date' : review_date,
                        'review_text' : review_text,
                        'review_photos' : photo_links,
                        'review_feedback' : review_data,
                        'check_ins' : user_check_ins,
                        'elite_status' : elite_status,
                        'previous_review' : previous_rating
                    }

                    review_list.append(user_information)
            except AttributeError as error:
                print(error)

    print(json.dumps(review_list, indent=4, sort_keys=True))

In [17]:
yelp_bakery_lookup(damen_bakeries, damen_reviews)

'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find_all'
[
    {
        "check_ins": 0,
        "elite_status": "elite",
        "previous_review": [],
        "review_date": "12/31/2018",
        "review_feedback": {
            "review_count": [
                "0",
                "1",
                "1"
            ],
            "review_type": [
                "Useful",
                "Funny",
                "Cool"
            ]
        },
        "review_photos": [],
        "review_text": "I adore this neighborhood bake

In [18]:
yelp_bakery_lookup(belmont_bakeries, belmont_reviews)

'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find'
'NoneT

In [19]:
yelp_bakery_lookup(logan_bakeries, logan_reviews)

'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find_all'
[
    {
        "check_ins": 1,
        "elite_status": "not elite",
        "previous_review": {
            "review_date": "10/8/2018\n            \nPrevious review",
            "review_feedback": {
                "review_count": [
                    "0",
                    "0",
                    "0"
                ],
                "review_type": [
                    "Useful",
                    "Funny",
                    "Cool"
                ]
            },
            "review_text": "This place has the BEST croissant I've had in Chicago! They're soft, crispy, and perfectly filled. This is a great spot for doing work and for catching up with friends. I've also had their quiche and macaroons. The quiche is really

In [23]:
print(len(damen_reviews))
print(len(belmont_reviews))
print(len(logan_reviews))

71
176
37


In [21]:
def rearrange_feedback(review_list):
    for review in review_list:
        counts = review['review_feedback']['review_count']
        review['useful_rating_count'] = counts[0]
        review['funny_rating_count'] = counts[1]
        review['cool_rating_count'] = counts[2]
    print(review_list[0])

In [22]:
rearrange_feedback(damen_reviews)
rearrange_feedback(belmont_reviews)
rearrange_feedback(logan_reviews)

{'user_name': 'Courtney J.', 'user_location': 'Chicago, IL', 'user_link': 'https://www.yelp.com/user_details?userid=aqkJ_0_ILP_myoK5-r2G-Q', 'user_friends': 263, 'user_reviews': 61, 'user_photos': 10, 'stars': 5, 'review_date': '12/31/2018', 'review_text': 'I adore this neighborhood bakery. It is a few blocks from where I live, so I find myself here on many weekend mornings. Everything is so fresh -- from the made to order sandwiches, to the freshly baked breads and pastries. The delicious smells from the kitchen and the large crowds say it all! I have only been here in the winter, but I imagine the patio would be a great place to enjoy a quick coffee and bite. The only suggestion I have is to potentially have a better system for keeping track of "to-go" orders. I\'ve waited quite a while for to-go items that were forgotten by the staff who took my order.', 'review_photos': [], 'review_feedback': {'review_type': ['Useful', 'Funny', 'Cool'], 'review_count': ['0', '1', '1']}, 'check_ins'

In [28]:
damen_review_df = pd.DataFrame(damen_reviews).set_index('user_name')
damen_review_df.head()

,check_ins,cool_rating_count,elite_status,funny_rating_count,previous_review,review_date,review_feedback,review_photos,review_text,stars,useful_rating_count,user_friends,user_link,user_location,user_photos,user_reviews
user_name,,,,,,,,,,,,,,,,
Courtney J.,0,1,elite,1,[],12/31/2018,"{'review_type': ['Useful', 'Funny', 'Cool'], '...",[],I adore this neighborhood bakery. It is a few ...,5,0,263,https://www.yelp.com/user_details?userid=aqkJ_...,"Chicago, IL",10,61
Sabrina G.,1,0,elite,0,[],12/23/2018,"{'review_type': ['Useful', 'Funny', 'Cool'], '...",[https://s3-media4.fl.yelpcdn.com/bphoto/SqCN8...,I've been dreaming of quiche and croissants as...,5,0,195,https://www.yelp.com/user_details?userid=LlGh8...,"Chicago, IL",82,51
Alfie B.,0,1,elite,0,[],10/10/2018,"{'review_type': ['Useful', 'Funny', 'Cool'], '...",[https://s3-media4.fl.yelpcdn.com/bphoto/oLYVK...,We came to La Boulangerie after recommendation...,5,0,16,https://www.yelp.com/user_details?userid=mymQs...,"Columbia, SC",323,106
Anna L.,0,1,elite,1,[],10/9/2018,"{'review_type': ['Useful', 'Funny', 'Cool'], '...",[],This is a great breakfast spot. Even better if...,5,1,28,https://www.yelp.com/user_details?userid=1aah9...,"Columbia, SC",145,176
Jordan E.,0,0,not elite,0,[],10/2/2018,"{'review_type': ['Useful', 'Funny', 'Cool'], '...",[],"Everytime i go here im like, wait a minute, is...",3,0,0,https://www.yelp.com/user_details?userid=0ANzg...,"Chicago, IL",6,20


In [29]:
damen_review_df.columns

Index(['check_ins', 'cool_rating_count', 'elite_status', 'funny_rating_count',
       'previous_review', 'review_date', 'review_feedback', 'review_photos',
       'review_text', 'stars', 'useful_rating_count', 'user_friends',
       'user_link', 'user_location', 'user_photos', 'user_reviews'],
      dtype='object')

In [30]:
damen_review_df = damen_review_df[['review_date', 'stars', 'user_location', 'review_text', 'check_ins', 'cool_rating_count', 'useful_rating_count', \
                       'funny_rating_count', 'review_photos', 'previous_review', 'elite_status', 'user_link', 'user_friends', \
                       'user_photos', 'user_reviews']]
damen_review_df.head()

,review_date,stars,user_location,review_text,check_ins,cool_rating_count,useful_rating_count,funny_rating_count,review_photos,previous_review,elite_status,user_link,user_friends,user_photos,user_reviews
user_name,,,,,,,,,,,,,,,
Courtney J.,12/31/2018,5,"Chicago, IL",I adore this neighborhood bakery. It is a few ...,0,1,0,1,[],[],elite,https://www.yelp.com/user_details?userid=aqkJ_...,263,10,61
Sabrina G.,12/23/2018,5,"Chicago, IL",I've been dreaming of quiche and croissants as...,1,0,0,0,[https://s3-media4.fl.yelpcdn.com/bphoto/SqCN8...,[],elite,https://www.yelp.com/user_details?userid=LlGh8...,195,82,51
Alfie B.,10/10/2018,5,"Columbia, SC",We came to La Boulangerie after recommendation...,0,1,0,0,[https://s3-media4.fl.yelpcdn.com/bphoto/oLYVK...,[],elite,https://www.yelp.com/user_details?userid=mymQs...,16,323,106
Anna L.,10/9/2018,5,"Columbia, SC",This is a great breakfast spot. Even better if...,0,1,1,1,[],[],elite,https://www.yelp.com/user_details?userid=1aah9...,28,145,176
Jordan E.,10/2/2018,3,"Chicago, IL","Everytime i go here im like, wait a minute, is...",0,0,0,0,[],[],not elite,https://www.yelp.com/user_details?userid=0ANzg...,0,6,20


In [31]:
damen_review_df.to_csv('damen_reviews.csv')

In [32]:
belmont_review_df = pd.DataFrame(belmont_reviews).set_index('user_name')
belmont_review_df = belmont_review_df[['review_date', 'stars', 'user_location', 'review_text', 'check_ins', 'cool_rating_count', 'useful_rating_count', \
                       'funny_rating_count', 'review_photos', 'previous_review', 'elite_status', 'user_link', 'user_friends', \
                       'user_photos', 'user_reviews']]
belmont_review_df.head()
belmont_review_df.to_csv('belmont_reviews.csv')


In [33]:
logan_review_df = pd.DataFrame(logan_reviews).set_index('user_name')
logan_review_df = logan_review_df[['review_date', 'stars', 'user_location', 'review_text', 'check_ins', 'cool_rating_count', 'useful_rating_count', \
                       'funny_rating_count', 'review_photos', 'previous_review', 'elite_status', 'user_link', 'user_friends', \
                       'user_photos', 'user_reviews']]
logan_review_df.head()
logan_review_df.to_csv('logan_reviews.csv')


In [34]:
school_link = 'https://www.lyceechicago.org/calendar?mini='
year_list = ['2016-07', '2016-08', '2016-09', '2016-10', '2016-11', '2016-12', '2017-01', '2017-02', '2017-03', \
             '2017-04', '2017-05', '2017-06', '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12', \
             '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09', \
             '2018-10', '2018-11']
event_list=[]

In [35]:
for month in year_list:
    new_link = school_link + month
    school_response = requests.get(new_link)
    school_soup = bs(school_response.text, 'html.parser')
    events = school_soup.find_all('td', class_='has-events')
    for e in events:
        event_link = e.find('div').find('a').attrs['href']
        event_list.append(event_link)

In [36]:
len(event_list)

153

In [37]:
print(event_list)

['https://www.lyceechicago.org/calendar/d/2016-08-22', 'https://www.lyceechicago.org/calendar/d/2016-08-23', 'https://www.lyceechicago.org/calendar/d/2016-08-24', 'https://www.lyceechicago.org/calendar/d/2016-08-25', 'https://www.lyceechicago.org/calendar/d/2016-08-26', 'https://www.lyceechicago.org/calendar/d/2016-08-29', 'https://www.lyceechicago.org/calendar/d/2016-08-30', 'https://www.lyceechicago.org/calendar/d/2016-08-31', 'https://www.lyceechicago.org/calendar/d/2016-09-01', 'https://www.lyceechicago.org/calendar/d/2016-09-05', 'https://www.lyceechicago.org/calendar/d/2016-09-09', 'https://www.lyceechicago.org/calendar/d/2016-09-10', 'https://www.lyceechicago.org/calendar/d/2016-09-26', 'https://www.lyceechicago.org/calendar/d/2016-10-17', 'https://www.lyceechicago.org/calendar/d/2016-10-18', 'https://www.lyceechicago.org/calendar/d/2016-10-19', 'https://www.lyceechicago.org/calendar/d/2016-10-20', 'https://www.lyceechicago.org/calendar/d/2016-10-21', 'https://www.lyceechicago.o

In [39]:
event_details = []
for event in event_list:
    date = event.split('/')
    date = date[len(date)-1]
    event_response = requests.get(event)
    event_soup = bs(event_response.text, 'html.parser')
    events_that_day = event_soup.find('div', class_='view-content').find_all('div', class_='views-row')
    for event in events_that_day:
        title = event.find('h3', class_='field-content').find('a').text.strip()
        time_frame = event.find('span', class_='date-display-single').text.strip()
        event_l = event.find('h3', class_='field-content').find('a').attrs['href']
        info = {
            'event_title' : title,
            'event_time' : time_frame,
            'event_date' : date,
            'event_link' : event_l
        }
        event_details.append(info)
print(json.dumps(event_details, indent=4, sort_keys=True))

[
    {
        "event_date": "2016-08-22",
        "event_link": "/calendar/back-to-school-prep-for-staff",
        "event_time": "(All day)",
        "event_title": "Back to School Prep for New Teachers"
    },
    {
        "event_date": "2016-08-23",
        "event_link": "/calendar/back-to-school-prep-for-staff",
        "event_time": "(All day)",
        "event_title": "Back to School Prep for New Teachers"
    },
    {
        "event_date": "2016-08-24",
        "event_link": "/calendar/back-to-school-prep-for-all-staff-0",
        "event_time": "(All day)",
        "event_title": "Back to School Prep for All Staff"
    },
    {
        "event_date": "2016-08-25",
        "event_link": "/calendar/back-to-school-prep-for-all-staff-0",
        "event_time": "(All day)",
        "event_title": "Back to School Prep for All Staff"
    },
    {
        "event_date": "2016-08-26",
        "event_link": "/calendar/back-to-school-prep-for-all-staff-0",
        "event_time": "(All day)",


In [40]:
print(len(event_details))
print(len(event_list))

153
153


In [41]:
for deet in event_details:
    deet['event_link'] = 'https://www.lyceechicago.org' + deet['event_link']
print(event_details)

[{'event_title': 'Back to School Prep for New Teachers', 'event_time': '(All day)', 'event_date': '2016-08-22', 'event_link': 'https://www.lyceechicago.org/calendar/back-to-school-prep-for-staff'}, {'event_title': 'Back to School Prep for New Teachers', 'event_time': '(All day)', 'event_date': '2016-08-23', 'event_link': 'https://www.lyceechicago.org/calendar/back-to-school-prep-for-staff'}, {'event_title': 'Back to School Prep for All Staff', 'event_time': '(All day)', 'event_date': '2016-08-24', 'event_link': 'https://www.lyceechicago.org/calendar/back-to-school-prep-for-all-staff-0'}, {'event_title': 'Back to School Prep for All Staff', 'event_time': '(All day)', 'event_date': '2016-08-25', 'event_link': 'https://www.lyceechicago.org/calendar/back-to-school-prep-for-all-staff-0'}, {'event_title': 'Back to School Prep for All Staff', 'event_time': '(All day)', 'event_date': '2016-08-26', 'event_link': 'https://www.lyceechicago.org/calendar/back-to-school-prep-for-all-staff-0'}, {'eve

In [42]:
lychee_event_df = pd.DataFrame(event_details)
lychee_event_df.head()

,event_date,event_link,event_time,event_title
0,2016-08-22,https://www.lyceechicago.org/calendar/back-to-...,(All day),Back to School Prep for New Teachers
1,2016-08-23,https://www.lyceechicago.org/calendar/back-to-...,(All day),Back to School Prep for New Teachers
2,2016-08-24,https://www.lyceechicago.org/calendar/back-to-...,(All day),Back to School Prep for All Staff
3,2016-08-25,https://www.lyceechicago.org/calendar/back-to-...,(All day),Back to School Prep for All Staff
4,2016-08-26,https://www.lyceechicago.org/calendar/back-to-...,(All day),Back to School Prep for All Staff


In [43]:
lychee_event_df.to_csv('lychee_events.csv')